In [1]:
# 패키지
import pandas as pd
import geopandas as gpd
from shapely import wkt

# 데이터 불러오기
path = "~/Dacon/서울시 자치구별 도보 네트워크 공간정보.csv"
df = pd.read_csv(path, encoding="cp949")
df['geometry'] = df['링크 WKT'].copy()
df.drop(columns=['링크 WKT', "노드링크 유형", "노드 WKT", "노드 ID",	"노드 유형 코드"], inplace=True)

print("Dimension of Data: ",df.shape)

# 공간정보 열에 NA가 있는 행 삭제
df = df.dropna(subset=['geometry'])

# 전체 도로 중 보행 가능만 필터
df['링크 유형 코드'] = df['링크 유형 코드'].astype(str).str[0:4]

txt = """1000
1001
1010
1011
1100
1101
1110
1111
0001
0010
0011"""
walker_LINK_CODE = txt.split('\n')
df = df[df["링크 유형 코드"].isin(walker_LINK_CODE)]

print("보행 가능 필터 후: ",df.shape)

# 공간정보 데이터로 바꾸기 gdf
df['geometry'] = df['geometry'].apply(wkt.loads)
gdf = gpd.GeoDataFrame(df, geometry='geometry', crs="epsg:4326")

Dimension of Data:  (491082, 18)
보행 가능 필터 후:  (278763, 18)


In [2]:
import networkx as nx
import geopandas as gpd
from shapely.geometry import Point, LineString

def remove_dead_ends(roads_gdf, min_connection=2):
    """
    막다른 골목을 제거하는 함수
    
    Parameters:
    roads_gdf: 도로 네트워크가 포함된 GeoDataFrame
    min_connection: 최소 연결 수 (기본값: 2, 즉 양방향 연결)
    
    Returns:
    GeoDataFrame: 막다른 골목이 제거된 도로 네트워크
    """
    # 네트워크 그래프 생성
    G = nx.Graph()
    
    # 노드 딕셔너리 생성 (좌표를 키로 사용)
    node_connections = {}
    
    # 모든 도로의 시작점과 끝점을 노드로 추가
    for idx, row in roads_gdf.iterrows():
        coords = list(row.geometry.coords)
        for coord in coords:
            # 소수점 6자리까지만 사용하여 부동소수점 오차 방지
            coord = tuple(round(x, 6) for x in coord)
            if coord not in node_connections:
                node_connections[coord] = set()
    
        # 연속된 좌표들을 엣지로 추가
        for i in range(len(coords)-1):
            coord1 = tuple(round(x, 6) for x in coords[i])
            coord2 = tuple(round(x, 6) for x in coords[i+1])
            node_connections[coord1].add(coord2)
            node_connections[coord2].add(coord1)
            G.add_edge(coord1, coord2)
    
    # 충분한 연결이 없는 노드 식별
    dead_end_nodes = {node for node, connections in node_connections.items() 
                     if len(connections) < min_connection}
    
    # 막다른 골목이 포함된 도로 식별
    roads_to_keep = []
    for idx, row in roads_gdf.iterrows():
        coords = list(row.geometry.coords)
        coords = [tuple(round(x, 6) for x in coord) for coord in coords]
        
        # 도로의 시작점과 끝점이 모두 막다른 골목이 아닌 경우만 유지
        if not (coords[0] in dead_end_nodes or coords[-1] in dead_end_nodes):
            roads_to_keep.append(idx)
    
    # 결과 생성
    filtered_roads = roads_gdf.loc[roads_to_keep].copy()
    
    # 연결 정보 추가
    filtered_roads['num_connections'] = filtered_roads.apply(
        lambda row: min(
            len(node_connections[tuple(round(x, 6) for x in row.geometry.coords[0])]),
            len(node_connections[tuple(round(x, 6) for x in row.geometry.coords[-1])])
        ),
        axis=1
    )
    
    return filtered_roads

# 사용 예시
# 막다른 골목 제거
connected_roads = remove_dead_ends(gdf, min_connection=2)

In [5]:
connected_roads.to_csv("connected_road.csv", index=False)

In [11]:
# 패키지
import pandas as pd
import geopandas as gpd
from shapely import wkt

# 데이터 불러오기
path = "/home/sungil/Byte-King-rawdata/connected_road.csv"
df = pd.read_csv(path)
df['geometry'] = df['geometry'].apply(wkt.loads)
gdf = gpd.GeoDataFrame(df, geometry='geometry', crs="epsg:4326")

In [26]:
gdf.head()
gdf.shape

(234329, 19)

In [ ]:
import geopandas as gpd
from shapely.geometry import LineString, Point
from shapely.ops import nearest_points
import numpy as np

def find_roads_along_path(roads_gdf, start_point, end_point, buffer_distance=0.001):
    """
    두 지점 사이의 직선 경로와 가까운 도로들을 찾는 함수
    
    Parameters:
    roads_gdf: 도로 네트워크가 포함된 GeoDataFrame
    start_point: 시작점 (Point)
    end_point: 도착점 (Point)
    buffer_distance: 직선 경로 주변 검색 반경 (degrees)
    
    Returns:
    GeoDataFrame: 선택된 도로들이 포함된 GeoDataFrame
    """
    # 두 지점을 잇는 직선 생성
    direct_path = LineString([start_point, end_point])
    
    # 직선 주변에 버퍼 생성
    path_buffer = direct_path.buffer(buffer_distance)
    
    # 버퍼 내에 있는 도로 선택
    selected_roads = roads_gdf[roads_gdf.geometry.intersects(path_buffer)]
    
    # 직선과의 거리를 계산하여 컬럼 추가
    selected_roads['distance_to_path'] = selected_roads.geometry.apply(
        lambda x: direct_path.distance(x)
    )
    
    # 거리순으로 정렬
    selected_roads = selected_roads.sort_values('distance_to_path')
    
    return selected_roads

# 사용 예시
# 시작점과 끝점 생성
start_point = Point(126.92853, 37.48422)  # 신림역
end_point = Point(126.91786, 37.48940)    # 보라매공원

# 함수 실행
direct_route_roads = find_roads_along_path(gdf, start_point, end_point)

In [27]:
from find_best_line import find_optimized_path
from shapely.geometry import Point
import warnings

warnings.filterwarnings('ignore')

# 사용 예시
# 시작점과 끝점 설정
start_point = Point(126.92853, 37.48422)  # 신림역
end_point = Point(126.91786, 37.48940)    # 보라매공원

# 최적화된 경로 찾기
optimized_route = find_optimized_path(gdf, start_point, end_point)

In [28]:
optimized_route

,링크 ID,링크 유형 코드,시작노드 ID,종료노드 ID,링크 길이,시군구코드,시군구명,읍면동코드,읍면동명,고가도로,...,교량,터널,육교,횡단보도,"공원,녹지",건물내,geometry,num_connections,path_order,total_length
182023,231533,1111,37478.0,89274.0,112.628,1162000000,관악구,1162010200,신림동,0.0,...,0.0,0.0,0,0,0.0,0.0,"LINESTRING (126.92259 37.48737, 126.92314 37.4...",3,0,0.001068
187153,76166,1111,110073.0,115009.0,12.958,1162000000,관악구,1162010200,신림동,0.0,...,0.0,0.0,0,0,0.0,0.0,"LINESTRING (126.92195 37.48818, 126.92204 37.4...",3,1,0.000129
182550,266505,1111,38853.0,38854.0,14.967,1162000000,관악구,1162010200,신림동,0.0,...,0.0,0.0,0,0,0.0,0.0,"LINESTRING (126.92807 37.48423, 126.92824 37.4...",2,2,0.000169
183446,267448,1111,89274.0,37559.0,152.195,1162000000,관악구,1162010200,신림동,0.0,...,0.0,0.0,0,0,0.0,0.0,"LINESTRING (126.92314 37.48646, 126.92360 37.4...",3,3,0.001527
187297,97954,1111,38850.0,38851.0,32.698,1162000000,관악구,1162010200,신림동,0.0,...,0.0,0.0,0,0,0.0,0.0,"LINESTRING (126.92733 37.48415, 126.92770 37.4...",3,4,0.000369
186147,86346,1111,115009.0,37478.0,93.844,1162000000,관악구,1162010200,신림동,0.0,...,0.0,0.0,0,0,0.0,0.0,"LINESTRING (126.92204 37.48809, 126.92259 37.4...",3,5,0.000908
180259,213310,1111,19169.0,110072.0,162.520,1162000000,관악구,1162010200,신림동,0.0,...,0.0,0.0,0,0,0.0,0.0,"LINESTRING (126.91947 37.48938, 126.92030 37.4...",3,6,0.001783
185897,171911,1111,38852.0,38853.0,18.589,1162000000,관악구,1162010200,신림동,0.0,...,0.0,0.0,0,0,0.0,0.0,"LINESTRING (126.92787 37.48422, 126.92807 37.4...",2,7,0.000210
187061,92538,1111,110017.0,110018.0,102.650,1162000000,관악구,1162010200,신림동,0.0,...,0.0,0.0,0,0,0.0,0.0,"LINESTRING (126.91939 37.48941, 126.91884 37.4...",3,8,0.001158
184631,79060,1111,37559.0,37528.0,32.332,1162000000,관악구,1162010200,신림동,0.0,...,1.0,0.0,0,0,0.0,0.0,"LINESTRING (126.92424 37.48541, 126.92460 37.4...",3,9,0.000363


In [30]:
optimized_route['링크 길이'].sum()

1199.146

In [31]:
import folium
from folium import PolyLine
from tqdm import tqdm

M = folium.Map(location=[37.484208,126.929676], zoom_start=17, tiles='cartodbpositron')

# GeoDataFrame의 각 LINESTRING을 지도에 추가
for idx, row in tqdm(optimized_route.iterrows()):
    # LINESTRING 좌표 추출
    coordinates = [(y, x) for x, y in row.geometry.coords]

    # LineString 추가
    folium.PolyLine(
        locations=coordinates,
        weight=2,
        color='red',
        opacity=1.0,
        popup=f"LineString ID: {idx}"  # 팝업 정보 (필요에 따라 수정)
    ).add_to(M)

M

23it [00:00, 6112.60it/s]
